<a href="https://colab.research.google.com/github/sahana-github/Machine-learning-Demo/blob/main/ML_Pipelines_with_ZenML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lesson 1.1: ML Pipelines with ZenML


In [ ]:
%pip install "zenml [server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2 #required for colab
%pip install ipython

# Restart the kernel using the following command:
import os
os._exit(0)

⠙ Installing integrations...


In [1]:
NGROK_TOKEN = "2b7iDypIKYd6hTnPDs7jvb9fTgs_4BrQZEpyAQ432XkfqBL2e"

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab(): #colab only setup
#install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ZenML Setup

In [3]:
!rm -rf .zen
!zenml init

Initializing the ZenML global configuration version to 0.67.0
⠋ Initializing ZenML repository at /content.
Creating database tables
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
Creating default stack in workspace default...
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠏ Initializing 

In [4]:
import numpy as np
from sklearn.base import ClassifierMixin # Changed import statement to correct module
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  digits= load_digits()
  data =digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size = 0.2, shuffle=False
  )
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc= model.score(X_test, y_test)
  print(f"Test accuracy: (test_acc)")

train_test()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Test accuracy: (test_acc)


In [5]:
from string import digits
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple
@step
def importer() -> Tuple[
  Annotated[np.ndarray, "X_train"],
  Annotated [np.ndarray, "X_test"],
  Annotated [np.ndarray, "y_train"],
  Annotated [np.ndarray, "y_test"],
]:
#Load the digits dataset as numpy arrays.
  digits=load_digits()
  data= digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test= train_test_split(
      data, digits.target, test_size= 0.2, shuffle=False
  )
  return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,  # Corrected variable name
    y_train: np.ndarray,  # Corrected variable name
) -> ClassifierMixin:  # Corrected return type
    """Trains an SVC model."""
    model = SVC(gamma=0.001)  # Corrected typo in gamma
    model.fit(X_train, y_train)
    return model


@step
def evaluator(
    X_test: np.ndarray,  # Corrected variable name
    y_test: np.ndarray,  # Corrected variable name
    model: ClassifierMixin,
) -> float:
    """Evaluates the model and prints the test accuracy."""
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")  # Corrected print statement
    return test_acc

In [6]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  """Links all the steps together in a pipeline."""
  X_train, X_test, y_train, y_test = importer()
  # Pass X_train directly to the trainer, removing the subtraction
  model = svc_trainer(X_train=X_train, y_train=y_train)
  evaluator(X_test=X_test, y_test=y_test, model=model)

Running ZenML Pipelines

In [7]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Registered new pipeline: digits_pipeline.
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Step importer has started.
Step importer has finished in 2.579s.
Step importer completed successfully.
Step svc_trainer has started.
Step svc_trainer has finished in 0.725s.
Step svc_trainer completed successfully.
Step evaluator has started.
Test accuracy: 0.9583333333333334
Step evaluator has finished in 0.605s.
Step evaluator completed successfully.
Pipeline run has finished in 5.058s.


In [13]:
!pip install zenml

In [15]:
!zenml up

Deploying a local ZenML server with name 'local'.
Calling start method...
⠋ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠙ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠹ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠸ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠼ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠴ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠦ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠧ Starting service 'LocalZenServer[6fe0853e-332c-4bc3-ac30-c660febb0d70] (type: zen_server, flavor: 
local)'.
⠇ Starting service 'LocalZenServer[6fe0853e-33

In [16]:
!pip install pyngrok
from pyngrok import ngrok

# Start the ngrok tunnel for the ZenML server port
public_url = ngrok.connect(8237)
print("ZenML dashboard available at:", public_url)


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-bd376d1c-fc0c-4d52-8e05-99a14b2a3ee7


Opening tunnel named: http-8237-bd376d1c-fc0c-4d52-8e05-99a14b2a3ee7


INFO:pyngrok.process.ngrok:t=2024-10-02T03:28:30+0000 lvl=info msg="no configuration paths supplied"


t=2024-10-02T03:28:30+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-10-02T03:28:30+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2024-10-02T03:28:30+0000 lvl=info msg=start pg=/api/tunnels id=4561e9c5c99cdc42


INFO:pyngrok.process.ngrok:t=2024-10-02T03:28:30+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-bd376d1c-fc0c-4d52-8e05-99a14b2a3ee7 addr=http://localhost:8237 url=https://a84c-35-196-225-121.ngrok-free.app


ZenML dashboard available at: NgrokTunnel: "https://a84c-35-196-225-121.ngrok-free.app" -> "http://localhost:8237"
t=2024-10-02T03:28:30+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-bd376d1c-fc0c-4d52-8e05-99a14b2a3ee7 addr=http://localhost:8237 url=https://a84c-35-196-225-121.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2024-10-02T03:28:30+0000 lvl=info msg=end pg=/api/tunnels id=4561e9c5c99cdc42 status=201 dur=44.668964ms


t=2024-10-02T03:28:30+0000 lvl=info msg=end pg=/api/tunnels id=4561e9c5c99cdc42 status=201 dur=44.668964ms
